<a href="https://colab.research.google.com/github/bbishop3/helloAI/blob/main/Assignment5_part1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow_decision_forests


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 50.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import tensorflow_decision_forests as tfdf

In [19]:
path = "car_evaluation.csv"
pandas_dataset = pd.read_csv(path)

# Display the first 3 examples.
pandas_dataset.head()

,vhigh,vhigh.1,2,2.1,small,low,unacc
0,vhigh,vhigh,2,2,small,med,unacc
1,vhigh,vhigh,2,2,small,high,unacc
2,vhigh,vhigh,2,2,med,low,unacc
3,vhigh,vhigh,2,2,med,med,unacc
4,vhigh,vhigh,2,2,med,high,unacc


In [31]:
label = "vhigh"

classes = list(pandas_dataset[label].unique())
print(f"Label classes: {classes}")


pandas_dataset[label] = pandas_dataset[label].map(classes.index)

Label classes: ['vhigh', 'high', 'med', 'low']


In [20]:
pandas_dataset.shape

(1727, 7)

In [21]:
pandas_dataset.dtypes

vhigh      object
vhigh.1    object
2          object
2.1        object
small      object
low        object
unacc      object
dtype: object

In [22]:
pandas_dataset.describe()

,vhigh,vhigh.1,2,2.1,small,low,unacc
count,1727,1727,1727,1727,1727,1727,1727
unique,4,4,4,3,3,3,4
top,high,high,3,4,med,med,unacc
freq,432,432,432,576,576,576,1209


In [23]:
pandas_dataset.isnull().mean()

vhigh      0.0
vhigh.1    0.0
2          0.0
2.1        0.0
small      0.0
low        0.0
unacc      0.0
dtype: float64

In [26]:
np.random.seed(1)
# Use the ~10% of the examples as the testing set
# and the remaining ~90% of the examples as the training set.
test_indices = np.random.rand(len(pandas_dataset)) < 0.1
pandas_train_dataset = pandas_dataset[~test_indices]
pandas_test_dataset = pandas_dataset[test_indices]

print("Training examples: ", len(pandas_train_dataset))
# >> Training examples: 309

print("Testing examples: ", len(pandas_test_dataset))
# >> Testing examples: 35

Training examples:  1554
Testing examples:  173


In [32]:
tf_train_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(pandas_train_dataset, label=label)
model = tfdf.keras.CartModel()
model.fit(tf_train_dataset)

Use /tmp/tmpdy40_z93 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.232030. Found 1554 examples.
Training model...
Model trained in 0:00:00.041315
Compiling model...
Model compiled.


In [33]:
tfdf.model_plotter.plot_model_in_colab(model, max_depth=10)


In [34]:
model.compile("accuracy")
print("Train evaluation: ", model.evaluate(tf_train_dataset, return_dict=True))
# >> Train evaluation:  {'loss': 0.0, 'accuracy': 0.96116}

tf_test_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(pandas_test_dataset, label=label)
print("Test evaluation: ", model.evaluate(tf_test_dataset, return_dict=True))
# >> Test evaluation:  {'loss': 0.0, 'accuracy': 0.97142}

2/2 [==============================] - 0s 9ms/step - loss: 0.0000e+00 - accuracy: 0.3520
Train evaluation:  {'loss': 0.0, 'accuracy': 0.3519948422908783}
1/1 [==============================] - 0s 122ms/step - loss: 0.0000e+00 - accuracy: 0.2370
Test evaluation:  {'loss': 0.0, 'accuracy': 0.23699422180652618}


In [35]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:
import keras_tuner as kt

def build_model(hp):
  model = tfdf.keras.CartModel(
      min_examples=hp.Choice("min_examples",
          # Try four possible values for "min_examples" hyperparameter.
          # min_examples=10 would limit the growth of the decision tree,
          # while min_examples=1 would lead to deeper decision trees.
         [1, 2, 5, 10]),
      validation_ratio=hp.Choice("validation_ratio",
         # Three possible values for the "validation_ratio" hyperparameter.
         [0.0, 0.05, 0.10]),
      )
  model.compile("accuracy")
  return model

tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=10,
    directory="/tmp/tuner",
    project_name="tune_cart")

tuner.search(x=tf_train_dataset, validation_data=tf_test_dataset)
best_model = tuner.get_best_models()[0]

print("Best hyperparameters: ", tuner.get_best_hyperparameters()[0].values)
# >> Best hyperparameters:  {'min_examples': 2, 'validation_ratio': 0.0}

Use /tmp/tmpkkvamksr as temporary training directory
Best hyperparameters:  {'min_examples': 2, 'validation_ratio': 0.05}


In [37]:
model = tfdf.keras.CartModel(min_examples=2, validation_ratio=0.0)
model.fit(tf_train_dataset)

model.compile("accuracy")
print("Test evaluation: ", model.evaluate(tf_test_dataset, return_dict=True))
# >> Test evaluation:  {'loss': 0.0, 'accuracy': 1.0}

Use /tmp/tmphzrs7po_ as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.578998. Found 1554 examples.
Training model...
Model trained in 0:00:00.145170
Compiling model...
Model compiled.
1/1 [==============================] - 1s 528ms/step - loss: 0.0000e+00 - accuracy: 0.0694
Test evaluation:  {'loss': 0.0, 'accuracy': 0.06936416029930115}


In [38]:
tfdf.model_plotter.plot_model_in_colab(model, max_depth=10)
